In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import librosa

import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

from IPython.display import Image

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from glob import glob


In [ ]:
# Helper Functions

In [35]:
def create_directories(sound_source, categories):
    for sound_source in sound_sources:
        for category in categories:
            os.makedirs(os.path.join('full_dataset',sound_source+'_data', 'train', category), exist_ok=True)
            os.makedirs(os.path.join('full_dataset',sound_source+'_data', 'val', category), exist_ok=True)
            os.makedirs(os.path.join('full_dataset',sound_source+'_data', 'test', category), exist_ok=True)
            


sound_sources = ['fan', 'pump', 'slider', 'valve']
classifications = ['normal', 'abnormal']

for sound_source in sound_sources:
    for classification in classifications:
        create_directories(sound_sources, classifications)


In [32]:
def save_mel_spectrogram(audio_vector, sr, save_path):
    # Set the figure size and DPI to get a 224x224 image
    fig_size = 224 / 100  # inches (224 pixels / 100 DPI)
    dpi = 100  # Dots per inch

    # Plot and save Mel spectrogram as an image
    fig, ax = plt.subplots(figsize=(fig_size, fig_size), dpi=dpi)
    ax.set_axis_off()
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove padding
    
    sgram = librosa.stft(audio_vector)  # extract short time fourier transform
    sgram_mag, _ = librosa.magphase(sgram)
    mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sr)
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)  # decibel scale
    
    librosa.display.specshow(mel_sgram, sr=sr, ax=ax)
    
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()

In [33]:
# Function to save Mel spectrogram images for a dataset
def save_spectrogram_images(X, y, sound_source, data_type, base_dir='full_dataset'):

    # Iterate through each sample
    for idx, audio_vector in enumerate(X):
        # Determine classification
        classification = 'normal' if y[idx] == 0 else 'abnormal'
        
        # Define file path
        file_path = os.path.join(base_dir, sound_source+'_data', data_type, classification, f'{idx}.png')

        # Save Mel spectrogram image
        save_mel_spectrogram(audio_vector, sr=16000, save_path=file_path)

In [27]:
def train_test_split_audio(audio_file_list, random_state=42):
    data = []
    labels = []
    
    for audio_file in audio_file_list:
        raw_data, sample_rate = librosa.load(audio_file, sr=None)
        classification = str(audio_file).split('/')[-2]
        
        if classification == 'normal':
            labels.append(0)
        else:
            labels.append(1) 
            
        data.append(raw_data)
    
    data = np.array(data)
    labels = np.array(labels)
    
    # Perform stratified split
    X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, stratify=labels, random_state=random_state)
    
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=random_state)
    
    return X_train, X_test, X_val, y_train, y_test, y_val

# Process Images

In [ ]:
### fans

In [20]:
fans = glob('/Users/sayyedjilani/datasci210/*_fan/*/normal/*.wav')
fans_abnormal = glob('/Users/sayyedjilani/datasci210/*_fan/*/abnormal/*.wav')

In [21]:
print(f'Normal fans: {len(fans)}')
print(f'Abormal fans: {len(fans_abnormal)}')

Normal fans: 12225
Abormal fans: 4425


In [22]:
fans.extend(fans_abnormal)

In [29]:
X_train_fan, X_test_fan, X_val_fan, y_train_fan, y_test_fan, y_val_fan = train_test_split_audio(fans, random_state=42)

In [36]:
save_spectrogram_images(X_train_fan, y_train_fan, 'fan', 'train')
save_spectrogram_images(X_val_fan, y_val_fan, 'fan', 'val')
save_spectrogram_images(X_test_fan, y_test_fan, 'fan', 'test')

In [ ]:
### pumps

In [37]:
pumps = glob('/Users/sayyedjilani/datasci210/*_pump/*/normal/*.wav')
pumps_abnormal = glob('/Users/sayyedjilani/datasci210/*_pump/*/abnormal/*.wav')

In [38]:
print(f'Normal pumps: {len(pumps)}')
print(f'Abormal pumps: {len(pumps_abnormal)}')

pumps.extend(pumps_abnormal)

Normal pumps: 11247
Abormal pumps: 1368


In [39]:
X_train_pump, X_test_pump, X_val_pump, y_train_pump, y_test_pump, y_val_pump = train_test_split_audio(pumps, random_state=42)

In [40]:
save_spectrogram_images(X_train_pump, y_train_pump, 'pump', 'train')
save_spectrogram_images(X_val_pump, y_val_pump, 'pump', 'val')
save_spectrogram_images(X_test_pump, y_test_pump, 'pump', 'test')

In [ ]:
### sliders

In [41]:
sliders = glob('/Users/sayyedjilani/datasci210/*_slider/*/normal/*.wav')
sliders_abnormal = glob('/Users/sayyedjilani/datasci210/*_slider/*/abnormal/*.wav')

In [42]:
print(f'Normal sliders: {len(sliders)}')
print(f'Abormal sliders: {len(sliders_abnormal)}')

sliders.extend(sliders_abnormal)

Normal sliders: 9612
Abormal sliders: 2670


In [43]:
X_train_slider, X_test_slider, X_val_slider, y_train_slider, y_test_slider, y_val_slider = train_test_split_audio(sliders, random_state=42)

In [44]:
save_spectrogram_images(X_train_slider, y_train_slider, 'slider', 'train')
save_spectrogram_images(X_val_slider, y_val_slider, 'slider', 'val')
save_spectrogram_images(X_test_slider, y_test_slider, 'slider', 'test')

In [ ]:
### valves

In [45]:
valves = glob('/Users/sayyedjilani/datasci210/*_valve/*/normal/*.wav')
valves_abnormal = glob('/Users/sayyedjilani/datasci210/*_valve/*/abnormal/*.wav')

In [46]:
print(f'Normal sliders: {len(valves)}')
print(f'Abormal sliders: {len(valves_abnormal)}')

valves.extend(valves_abnormal)

Normal sliders: 11073
Abormal sliders: 1437


In [47]:
X_train_valve, X_test_valve, X_val_valve, y_train_valve, y_test_valve, y_val_valve = train_test_split_audio(valves, random_state=42)

In [48]:
save_spectrogram_images(X_train_valve, y_train_valve, 'valve', 'train')
save_spectrogram_images(X_val_valve, y_val_valve, 'valve', 'val')
save_spectrogram_images(X_test_valve, y_test_valve, 'valve', 'test')